In [14]:
API_KEY = '70b3c2c2b36636ca97ae5337ca597afa'
USER_AGENT = 'Music Recommender System'

In [61]:
import requests

In [12]:
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

In [32]:
r = lastfm_get({
    'method': 'user.getFriends',
    'user' : "rj"
})

In [12]:
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

jprint(r.json())

{
    "lovedtracks": {
        "@attr": {
            "page": "1",
            "perPage": "50",
            "total": "700",
            "totalPages": "14",
            "user": "RJ"
        },
        "track": [
            {
                "artist": {
                    "mbid": "dc76f538-7a10-461e-84ba-aa44888f9f2d",
                    "name": "The Black Seeds",
                    "url": "https://www.last.fm/music/The+Black+Seeds"
                },
                "date": {
                    "#text": "04 Apr 2016, 10:53",
                    "uts": "1459767227"
                },
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "medium"
 

In [1]:
import time
import re
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
import progressbar
import pylast
import os
import operator
import scipy.sparse as sp
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
API_KEY = "70b3c2c2b36636ca97ae5337ca597afa"
API_SECRET = "ad165f97b770cb95b5c9b325aff8e8b6"
PATH_USERNAMES = "../Data/usernames.npy"
# PATH_SONGS = "dataset/songs.npy"

In [3]:
def get_username_from_url(request):
    res = set()
    with urllib.request.urlopen(request) as urlObject:
        html = urlObject.read()
        soup = BeautifulSoup(html, "html.parser")
        results = soup.findAll('li', { "class" : "b_algo" })
        for result in results:
            result = str(result.find('a').get('href'))
            username = re.search(r'https://[^.]+.last.fm/user/([^/]+)', result)
            if username is not None:
                res.add(username.group(1))
            else:
                print('failed to fetch username in', result)
    return res

In [4]:
def bing_search():
    print('crawling usernames from Bing')
    url = "https://www.bing.com/search?q=site%3alast.fm%2fuser%2f&qs=ds&first={0}&FORM=PERE"
    headers = {'User-Agent':'Chro/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'}
    res = set()
    start = 1
    address = url.format(start)
    request = urllib.request.Request(address, None, headers)
    res = res.union(get_username_from_url(request))
    start = 15
    total = 1000
    with progressbar.ProgressBar(max_value=total) as bar:
        while start < total:
            address = url.format(start)
            request = urllib.request.Request(address, None, headers)     
            res = res.union(get_username_from_url(request))
            start += 10
            bar.update(min(start, total))
            time.sleep(1)
    print('crawled %d usernames from bing search'%len(res))
    return res

In [5]:
network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET)

def crawl_user_friends(usernames, depth=1, max_friends=0):
    usernames = usernames[:]
    for i in range(depth):
        print('fetching list of usernames with a depth of %d'%depth)
        assert len(usernames) == i+1
        friends = set()
        # traverse each user in last depth
        with progressbar.ProgressBar(max_value=len(usernames[i])) as bar:
            for j, username in enumerate(usernames[i]):
                user = network.get_user(username)
                try:
                    n = 0
                    for friend in user.get_friends():
                        recorded = False
                        for prev in usernames: # check if friend is already recorded in previous depths
                            if friend.get_name() in prev:
                                recorded = True
                        if not recorded:
                            friends.add(friend.get_name())
                            n += 1
                        if max_friends and n >= max_friends:
                            break
                except:
                    pass
                finally:
                    bar.update(j)
        usernames.append(friends)
    return usernames

In [6]:
def get_usernames():
    path = os.path.join(os.getcwd(), PATH_USERNAMES)
    if os.path.isfile(path):
        return np.load(path, allow_pickle=True)
    dataset = crawl_user_friends([bing_search()], depth=1, max_friends=20)
    dataset = np.array(list(set.union(*dataset)), dtype=object)
    np.save(path, dataset)
    return dataset

In [7]:
USERNAMES = get_usernames()

crawling usernames from Bing


100% (1000 of 1000) |####################| Elapsed Time: 0:02:27 Time:  0:02:27
  0% (0 of 686) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

crawled 686 usernames from bing search
fetching list of usernames with a depth of 1


100% (686 of 686) |######################| Elapsed Time: 0:07:48 Time:  0:07:48


In [9]:
len(USERNAMES)

7093

In [10]:
USERNAMES[0]

'arierocknroll'

In [19]:
res = r.json()['lovedtracks']['track']
[(i["name"], i["artist"]['name'], i["date"]["#text"]) for i in res]

[('Kasih Jangan Pergi', 'Bunga', '04 Sep 2016, 22:21'),
 ('Fall Together', 'The Temper Trap', '04 Sep 2016, 22:09'),
 ('Menjadi Manusia', 'Bangkutaman', '04 Sep 2016, 12:09'),
 ('Dánarfregnir og Jarðarfarir', 'Sigur Rós', '27 Jan 2014, 00:57'),
 ('Let Me Love, Let Me Live', "Aphrodite's Child", '17 Jun 2013, 00:19'),
 ('25 Rasul', 'Raihan', '16 Jun 2013, 08:04'),
 ('Cahaya Selawat', 'Raihan', '16 Jun 2013, 08:04'),
 ('Whiskey Women (Acoustic)', 'Gugun Power Trio', '12 Jun 2013, 23:19'),
 ('Somebody', 'Aerosmith', '07 Jun 2013, 00:10'),
 ('Mama Kin', 'Aerosmith', '06 Jun 2013, 07:12'),
 ('Make It', 'Aerosmith', '06 Jun 2013, 07:09'),
 ('Lowdown', 'My Morning Jacket', '26 May 2013, 21:13'),
 ('I Am the Resurrection', 'The Stone Roses', '26 May 2013, 21:06'),
 ('Irresponsible Hate Anthem', 'Marilyn Manson', '30 Nov 2012, 22:48'),
 ('Skyfall', 'Adele', '30 Nov 2012, 22:45'),
 ('Miracle', 'Bon Jovi', '30 Nov 2012, 22:43'),
 ('Connjur', 'School of Seven Bells', '26 Nov 2012, 22:03'),
 ('Bad 

In [44]:
def get_playlist(user):
    
    r = lastfm_get({
    'method': 'user.getlovedtracks',
    'user' : user
    })
    try:
        res = r.json()['lovedtracks']['track']
    except:
        res = None
        return None
    
    return [(i["name"], i["artist"]['name'], i["date"]["#text"]) for i in res]

In [45]:
USERNAMES

array(['arierocknroll', 'deedee1whoa', 'Cashflo13_', ...,
       'SpicyRuddyDrops', 'parspilejums', 'porpinao'], dtype=object)

In [51]:
from tqdm import tqdm
import json
playlist_dict = dict()
playlist_count_dict = dict()
with open("../Data/playlist.jsonl", 'w') as f:
    for user in tqdm(USERNAMES):
        playlist = get_playlist(user)
        if playlist == None:
            pass
        out_row = dict()
        out_row["user_name"] = user
        out_row["playlist"] = playlist
        playlist_dict[user] = playlist
        try:
            playlist_count_dict[user] = len(playlist)
        except:
            playlist_count_dict[user] = 0
        f.write(json.dumps(out_row) + "\n")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7093/7093 [1:10:54<00:00,  1.67it/s]


In [52]:
len(playlist_dict)

7093

In [53]:
len(USERNAMES)

7093

In [56]:
from collections import Counter

In [59]:
Counter(playlist_count_dict.values())

Counter({50: 3652,
         0: 1772,
         11: 37,
         28: 27,
         7: 45,
         29: 22,
         39: 17,
         5: 70,
         38: 18,
         31: 17,
         10: 47,
         37: 15,
         35: 23,
         48: 13,
         18: 39,
         46: 20,
         3: 78,
         2: 97,
         1: 226,
         23: 23,
         22: 27,
         47: 21,
         49: 11,
         6: 49,
         12: 36,
         8: 48,
         27: 18,
         21: 31,
         33: 21,
         14: 31,
         40: 16,
         15: 31,
         24: 31,
         44: 23,
         4: 70,
         25: 27,
         9: 36,
         41: 17,
         17: 29,
         19: 33,
         43: 18,
         32: 18,
         42: 20,
         26: 18,
         45: 17,
         20: 27,
         36: 16,
         13: 31,
         34: 17,
         30: 15,
         16: 32})